In [1]:
# -*- coding: utf-8 -*-

"""
EVCのためのEV-GMMを構築します. そして, 適応学習する.
詳細 : https://pdfs.semanticscholar.org/cbfe/71798ded05fb8bf8674580aabf534c4dbb8bc.pdf

This program make EV-GMM for EVC. Then, it make adaptation learning.
Check detail : https://pdfs.semanticscholar.org/cbfe/71798ded05fb8bf8674580abf534c4dbb8bc.pdf
"""

'\nEVCのためのEV-GMMを構築します. そして, 適応学習する.\n詳細 : https://pdfs.semanticscholar.org/cbfe/71798ded05fb8bf8674580aabf534c4dbb8bc.pdf\n\nThis program make EV-GMM for EVC. Then, it make adaptation learning.\nCheck detail : https://pdfs.semanticscholar.org/cbfe/71798ded05fb8bf8674580abf534c4dbb8bc.pdf\n'

In [2]:
from __future__ import division, print_function

import os
from shutil import rmtree
import argparse
import glob
import pickle
import time

import numpy as np
from numpy.linalg import norm 
from sklearn.decomposition import PCA
from sklearn.mixture import GMM # sklearn 0.20.0から使えない
from sklearn.preprocessing import StandardScaler
import scipy.signal
import scipy.sparse

%matplotlib inline
import matplotlib.pyplot as plt
import IPython 
from IPython.display import Audio 

import soundfile as sf
import wave 
import pyworld as pw
import librosa.display

from dtw import dtw
import warnings
warnings.filterwarnings('ignore')

In [3]:
"""
Parameters

__Mixtured : GMM混合数
__versions : 実験セット
__convert_source : 変換元話者のパス
__convert_target : 変換先話者のパス
"""
# parameters 
__Mixtured = 40
__versions = 'pre-stored0.1.3'
__convert_source = 'input/EJM10/V01/T01/TIMIT/000/*.wav' 
__convert_target = 'adaptation/EJM05/V01/T01/ATR503/A/*.wav'

# settings
__same_path = './utterance/' + __versions + '/'
__output_path = __same_path + 'output/EJM05/' # EJF01, EJF07, EJM04, EJM05

Mixtured = __Mixtured
pre_stored_pickle = __same_path + __versions + '.pickle'
pre_stored_source_list = __same_path + 'pre-source/**/V01/T01/**/*.wav'
pre_stored_list = __same_path + "pre/**/V01/T01/**/*.wav"
#pre_stored_target_list = "" (not yet)
pre_stored_gmm_init_pickle = __same_path + __versions + '_init-gmm.pickle'
pre_stored_sv_npy = __same_path + __versions + '_sv.npy'

save_for_evgmm_covarXX = __output_path + __versions + '_covarXX.npy'
save_for_evgmm_covarYX = __output_path + __versions + '_covarYX.npy'
save_for_evgmm_fitted_source = __output_path + __versions + '_fitted_source.npy'
save_for_evgmm_fitted_target = __output_path + __versions + '_fitted_target.npy'
save_for_evgmm_weights = __output_path + __versions + '_weights.npy'
save_for_evgmm_source_means = __output_path + __versions + '_source_means.npy'

for_convert_source = __same_path + __convert_source
for_convert_target = __same_path + __convert_target
converted_voice_npy = __output_path + 'sp_converted_' + __versions  
converted_voice_wav = __output_path + 'sp_converted_' + __versions 
mfcc_save_fig_png = __output_path + 'mfcc3dim_' + __versions 
f0_save_fig_png = __output_path + 'f0_converted' + __versions
converted_voice_with_f0_wav = __output_path + 'sp_f0_converted' + __versions

In [4]:
EPSILON = 1e-8

class MFCC:
    """
    MFCC() : メル周波数ケプストラム係数(MFCC)を求めたり、MFCCからスペクトルに変換したりするクラス.
    動的特徴量(delta)が実装途中.
    ref : http://aidiary.hatenablog.com/entry/20120225/1330179868
    """
    
    
    def __init__(self, frequency, nfft=1026, dimension=24, channels=24):
        """
        各種パラメータのセット
        nfft : FFTのサンプル点数
        frequency : サンプリング周波数
        dimension : MFCC次元数
        channles : メルフィルタバンクのチャンネル数(dimensionに依存)
        fscale : 周波数スケール軸
        filterbankl, fcenters : フィルタバンク行列, フィルタバンクの頂点(?)
        """
        self.nfft = nfft
        self.frequency = frequency
        self.dimension = dimension
        self.channels = channels
        self.fscale = np.fft.fftfreq(self.nfft, d = 1.0 / self.frequency)[: int(self.nfft / 2)]
        self.filterbank, self.fcenters = self.melFilterBank()
    
    def hz2mel(self, f):
        """
        周波数からメル周波数に変換
        """
        return 1127.01048 * np.log(f / 700.0 + 1.0)
    
    def mel2hz(self, m):
        """
        メル周波数から周波数に変換
        """     
        return 700.0 * (np.exp(m / 1127.01048) - 1.0)

    def melFilterBank(self):
        """
        メルフィルタバンクを生成する
        """      
        fmax = self.frequency / 2
        melmax = self.hz2mel(fmax)
        nmax = int(self.nfft / 2)
        df = self.frequency / self.nfft
        dmel = melmax / (self.channels + 1)
        melcenters = np.arange(1, self.channels + 1) * dmel
        fcenters = self.mel2hz(melcenters)
        indexcenter = np.round(fcenters / df)
        indexstart = np.hstack(([0], indexcenter[0:self.channels - 1]))
        indexstop = np.hstack((indexcenter[1:self.channels], [nmax]))

        filterbank = np.zeros((self.channels, nmax))
        for c in np.arange(0, self.channels):
            increment = 1.0 / (indexcenter[c] - indexstart[c])
            # np,int_ は np.arangeが[0. 1. 2. ..]となるのをintにする
            for i in np.int_(np.arange(indexstart[c], indexcenter[c])):
                filterbank[c, i] = (i - indexstart[c]) * increment
            decrement = 1.0 / (indexstop[c] - indexcenter[c])
            # np,int_ は np.arangeが[0. 1. 2. ..]となるのをintにする
            for i in np.int_(np.arange(indexcenter[c], indexstop[c])):
                filterbank[c, i] = 1.0 - ((i - indexcenter[c]) * decrement)

        return filterbank, fcenters
    
    def mfcc(self, spectrum):
        """
        スペクトルからMFCCを求める.
        """
        mspec = []
        mspec = np.log10(np.dot(spectrum, self.filterbank.T))
        mspec = np.array(mspec)
        
        return scipy.fftpack.realtransforms.dct(mspec, type=2, norm="ortho", axis=-1)
    
    def delta(self, mfcc):
        """
        MFCCから動的特徴量を求める.
        現在は,求める特徴量フレームtをt-1とt+1の平均としている.
        """
        mfcc = np.concatenate([
            [mfcc[0]], 
            mfcc, 
            [mfcc[-1]]
        ]) # 最初のフレームを最初に、最後のフレームを最後に付け足す
        delta = None
        for i in range(1, mfcc.shape[0] - 1):
            slope = (mfcc[i+1] - mfcc[i-1]) / 2
            if delta is None:
                delta = slope
            else:
                delta = np.vstack([delta, slope])
        return delta
    
    def imfcc(self, mfcc, spectrogram):
        """
        MFCCからスペクトルを求める.
        """
        im_sp = np.array([])
        for i in range(mfcc.shape[0]):
            mfcc_s = np.hstack([mfcc[i], [0] * (self.channels - self.dimension)])
            mspectrum = scipy.fftpack.idct(mfcc_s, norm='ortho')
            # splrep はスプライン補間のための補間関数を求める
            tck = scipy.interpolate.splrep(self.fcenters, np.power(10, mspectrum))
            # splev は指定座標での補間値を求める
            im_spectrogram = scipy.interpolate.splev(self.fscale, tck)
            im_sp = np.concatenate((im_sp, im_spectrogram), axis=0)
            
        return im_sp.reshape(spectrogram.shape)
            
    def trim_zeros_frames(x, eps=1e-7):
        """
        無音区間を取り除く.
        """
        T, D = x.shape
        s = np.sum(np.abs(x), axis=1)
        s[s < 1e-7] = 0.
        return x[s > eps]

In [5]:
def analyse_by_world_with_harverst(x, fs):
    """
    WORLD音声分析合成器で基本周波数F0,スペクトル包絡,非周期成分を求める.
    基本周波数F0についてはharvest法により,より精度良く求める.
    """
    # 4 Harvest with F0 refinement (using Stonemask)
    frame_period = 5
    _f0_h, t_h = pw.harvest(x, fs, frame_period=frame_period)
    f0_h = pw.stonemask(x, _f0_h, t_h, fs)
    sp_h = pw.cheaptrick(x, f0_h, t_h, fs)
    ap_h = pw.d4c(x, f0_h, t_h, fs)
    
    return f0_h, sp_h, ap_h

def wavread(file):
    """
    wavファイルから音声トラックとサンプリング周波数を抽出する.
    """
    wf = wave.open(file, "r")
    fs = wf.getframerate()
    x = wf.readframes(wf.getnframes())
    x = np.frombuffer(x, dtype= "int16") / 32768.0
    wf.close()
    return x, float(fs)

def preEmphasis(signal, p=0.97):
    """
    MFCC抽出のための高域強調フィルタ.
    波形を通すことで,高域成分が強調される.
    """
    return scipy.signal.lfilter([1.0, -p], 1, signal)

def alignment(source, target, path):
    """
    タイムアライメントを取る.
    target音声をsource音声の長さに合うように調整する.
    """
    # ここでは814に合わせよう(targetに合わせる)
    # p_p = 0 if source.shape[0] > target.shape[0] else 1

    #shapes = source.shape if source.shape[0] > target.shape[0] else target.shape 
    shapes = source.shape
    align = np.array([])
    for (i, p) in enumerate(path[0]):
        if i != 0:
            if j != p:
                temp = np.array(target[path[1][i]])
                align = np.concatenate((align, temp), axis=0)
        else:
            temp = np.array(target[path[1][i]])
            align = np.concatenate((align, temp), axis=0)   
        
        j = p
        
    return align.reshape(shapes)

In [6]:
"""
pre-stored学習のためのパラレル学習データを作る。
時間がかかるため、利用できるlearn-data.pickleがある場合はそれを利用する。
それがない場合は一から作り直す。
"""
timer_start = time.time()
if os.path.exists(pre_stored_pickle):
    print("exist, ", pre_stored_pickle)
    with open(pre_stored_pickle, mode='rb') as f:
        total_data = pickle.load(f)
        print("open, ", pre_stored_pickle)
        print("Load pre-stored time = ", time.time() - timer_start , "[sec]")
else:
    source_mfcc = []
    #source_data_sets = []
    for name in sorted(glob.iglob(pre_stored_source_list, recursive=True)):
        print(name)
        x, fs = sf.read(name)
        f0, sp, ap = analyse_by_world_with_harverst(x, fs)
    
        mfcc = MFCC(fs)
        source_mfcc_temp = mfcc.mfcc(sp)
        #source_data = np.hstack([source_mfcc_temp, mfcc.delta(source_mfcc_temp)]) # static & dynamic featuers
        source_mfcc.append(source_mfcc_temp)
        #source_data_sets.append(source_data)

    total_data = []
    
    i = 0
    _s_len = len(source_mfcc)
    for name in sorted(glob.iglob(pre_stored_list, recursive=True)):
        print(name, len(total_data))
        x, fs = sf.read(name)
        f0, sp, ap = analyse_by_world_with_harverst(x, fs)

        mfcc = MFCC(fs)
        target_mfcc = mfcc.mfcc(sp)

        dist, cost, acc, path = dtw(source_mfcc[i%_s_len], target_mfcc, dist=lambda x, y: norm(x - y, ord=1))
        #print('Normalized distance between the two sounds:' + str(dist))
        #print("target_mfcc = {0}".format(target_mfcc.shape))

        aligned = alignment(source_mfcc[i%_s_len], target_mfcc, path)
        #target_data_sets = np.hstack([aligned, mfcc.delta(aligned)]) # static & dynamic features
        #learn_data = np.hstack((source_data_sets[i], target_data_sets))
        learn_data = np.hstack([source_mfcc[i%_s_len], aligned])

        total_data.append(learn_data)
        i += 1
        
    with open(pre_stored_pickle, 'wb') as output:
        pickle.dump(total_data, output)
        print("Make, ", pre_stored_pickle)
        print("Make pre-stored time = ", time.time() - timer_start , "[sec]")

./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A03.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A04.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A07.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A08.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A09.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A11.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A13.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A14.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A17.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A18.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A19.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A20.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A21.wav
./utterance/

./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/ESUS/000/A17.wav 72
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/ESUS/000/A18.wav 73
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/ESUS/000/A25.wav 74
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/ESUS/000/A29.wav 75
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/ESUS/000/A30.wav 76
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/ESUS/000/A39.wav 77
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/TIMIT/000/A01.wav 78
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/TIMIT/000/A02.wav 79
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/TIMIT/000/A03.wav 80
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/TIMIT/000/A04.wav 81
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/TIMIT/000/A05.wav 82
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/TIMIT/000/A06.wav 83
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/TIMIT/000/A07.wav 84
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/TIMIT/000/A08.wav 85
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/TIMIT/000/A09.wav 86
.

./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESPBOBAMA1/000/A14.wav 191
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESPBOBAMA1/000/A17.wav 192
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESPBOBAMA1/000/A18.wav 193
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESPBOBAMA1/000/A19.wav 194
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESPBOBAMA1/000/A20.wav 195
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESPBOBAMA1/000/A21.wav 196
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESPBOBAMA1/000/A22.wav 197
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESUS/000/A02.wav 198
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESUS/000/A03.wav 199
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESUS/000/A05.wav 200
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESUS/000/A06.wav 201
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESUS/000/A07.wav 202
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESUS/000/A08.wav 203
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESUS/000/A09.wav 204
./utterance/pre-stor

./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A03.wav 310
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A04.wav 311
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A05.wav 312
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A06.wav 313
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A07.wav 314
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A08.wav 315
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A09.wav 316
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A10.wav 317
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A12.wav 318
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A13.wav 319
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A15.wav 320
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A16.wav 321
./utterance/pre-stored0.1.2/pre/EJF10/V01/T01/ESPBOBAMA1/000/A03.wav 322
./utterance/pre-stored0.1.2/pre/EJF10/V01/T01/ESPBOBAMA1/000/A04.wav 323
./utterance/pre-stored0.1.2/pre/EJF10/

./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A02.wav 428
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A03.wav 429
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A05.wav 430
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A06.wav 431
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A07.wav 432
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A08.wav 433
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A09.wav 434
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A10.wav 435
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A11.wav 436
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A13.wav 437
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A14.wav 438
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A16.wav 439
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A17.wav 440
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A18.wav 441
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/E

./utterance/pre-stored0.1.2/pre/EJM01/V01/T01/TIMIT/000/A10.wav 547
./utterance/pre-stored0.1.2/pre/EJM01/V01/T01/TIMIT/000/A12.wav 548
./utterance/pre-stored0.1.2/pre/EJM01/V01/T01/TIMIT/000/A13.wav 549
./utterance/pre-stored0.1.2/pre/EJM01/V01/T01/TIMIT/000/A15.wav 550
./utterance/pre-stored0.1.2/pre/EJM01/V01/T01/TIMIT/000/A16.wav 551
./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESPBOBAMA1/000/A03.wav 552
./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESPBOBAMA1/000/A04.wav 553
./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESPBOBAMA1/000/A07.wav 554
./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESPBOBAMA1/000/A08.wav 555
./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESPBOBAMA1/000/A09.wav 556
./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESPBOBAMA1/000/A11.wav 557
./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESPBOBAMA1/000/A13.wav 558
./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESPBOBAMA1/000/A14.wav 559
./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESPBOBAMA1/000/A17.wav 560
./u

./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A11.wav 666
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A13.wav 667
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A14.wav 668
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A16.wav 669
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A17.wav 670
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A18.wav 671
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A25.wav 672
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A29.wav 673
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A30.wav 674
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A39.wav 675
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/TIMIT/000/A01.wav 676
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/TIMIT/000/A02.wav 677
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/TIMIT/000/A03.wav 678
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/TIMIT/000/A04.wav 679
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/TIMIT/000/A0

./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A08.wav 785
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A09.wav 786
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A11.wav 787
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A13.wav 788
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A14.wav 789
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A17.wav 790
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A18.wav 791
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A19.wav 792
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A20.wav 793
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A21.wav 794
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A22.wav 795
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESUS/000/A02.wav 796
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESUS/000/A03.wav 797
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESUS/000/A05.wav 

./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/ESUS/000/A29.wav 903
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/ESUS/000/A30.wav 904
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/ESUS/000/A39.wav 905
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A01.wav 906
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A02.wav 907
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A03.wav 908
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A04.wav 909
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A05.wav 910
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A06.wav 911
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A07.wav 912
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A08.wav 913
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A09.wav 914
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A10.wav 915
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A12.wav 916
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT

In [7]:
"""
全事前学習出力話者からラムダを推定する.
ラムダは適応学習で変容する.
"""

S = len(total_data)
D = int(total_data[0].shape[1] / 2)
print("total_data[0].shape = ", total_data[0].shape)
print("S = ", S)
print("D = ", D)

timer_start = time.time()
if os.path.exists(pre_stored_gmm_init_pickle):
    print("exist, ", pre_stored_gmm_init_pickle)
    with open(pre_stored_gmm_init_pickle, mode='rb') as f:
        initial_gmm = pickle.load(f)
        print("open, ", pre_stored_gmm_init_pickle)
        print("Load initial_gmm time = ", time.time() - timer_start , "[sec]")  
else:
    initial_gmm = GMM(n_components = Mixtured, covariance_type = 'full')
    initial_gmm.fit(np.vstack(total_data))
    with open(pre_stored_gmm_init_pickle, 'wb') as output:
        pickle.dump(initial_gmm, output)
        print("Make, ", initial_gmm)
        print("Make initial_gmm time = ", time.time() - timer_start , "[sec]")    
            
weights = initial_gmm.weights_
source_means = initial_gmm.means_[:, :D]
target_means = initial_gmm.means_[:, D:]
covarXX = initial_gmm.covars_[:, :D, :D]
covarXY = initial_gmm.covars_[:, :D, D:]
covarYX = initial_gmm.covars_[:, D:, :D]
covarYY = initial_gmm.covars_[:, D:, D:]

fitted_source = source_means
fitted_target = target_means

total_data[0].shape =  (1641, 48)
S =  1012
D =  24
Make,  GMM(covariance_type='full', init_params='wmc', min_covar=0.001,
  n_components=40, n_init=1, n_iter=100, params='wmc', random_state=None,
  tol=0.001, verbose=0)
Make initial_gmm time =  3966.042662858963 [sec]


In [8]:
"""
SVはGMMスーパーベクトルで、各pre-stored学習における出力話者について平均ベクトルを推定する。
GMMの学習を見てみる必要があるか？
"""

timer_start = time.time()
if os.path.exists(pre_stored_sv_npy):
    print("exist, ", pre_stored_sv_npy)
    sv = np.load(pre_stored_sv_npy)
    print("open, ", pre_stored_sv_npy)
    print("Load pre_stored_sv time = ", time.time() - timer_start , "[sec]")  
    
else:
    sv = []
    for i in range(S):
        gmm = GMM(n_components = Mixtured, params = 'm', init_params = '', covariance_type = 'full')
        gmm.weights_ = initial_gmm.weights_
        gmm.means_ = initial_gmm.means_
        gmm.covars_ = initial_gmm.covars_
        gmm.fit(total_data[i])
        sv.append(gmm.means_)
    sv = np.array(sv)
    np.save(pre_stored_sv_npy, sv)
    print("Make pre_stored_sv time = ", time.time() - timer_start , "[sec]")  

Make pre_stored_sv time =  305.34649109840393 [sec]


In [9]:
"""
各事前学習出力話者のGMM平均ベクトルに対して主成分分析(PCA)を行う.
PCAで求めた固有値と固有ベクトルからeigenvectorsとbiasvectorsを作る.
"""
timer_start = time.time()
#source_pca
source_n_component, source_n_features = sv[:, :, :D].reshape(S, Mixtured*D).shape
# 標準化(分散を1、平均を0にする)
source_stdsc = StandardScaler()
# 共分散行列を求める
source_X_std = source_stdsc.fit_transform(sv[:, :, :D].reshape(S, Mixtured*D)) 

# PCAを行う
source_cov = source_X_std.T @ source_X_std / (source_n_component - 1)
source_W, source_V_pca = np.linalg.eig(source_cov)

print(source_W.shape)
print(source_V_pca.shape)

# データを主成分の空間に変換する
source_X_pca = source_X_std @ source_V_pca
print(source_X_pca.shape)

#target_pca
target_n_component, target_n_features = sv[:, :, D:].reshape(S, Mixtured*D).shape
# 標準化(分散を1、平均を0にする)
target_stdsc = StandardScaler()
#共分散行列を求める
target_X_std = target_stdsc.fit_transform(sv[:, :, D:].reshape(S, Mixtured*D)) 

#PCAを行う
target_cov = target_X_std.T @ target_X_std / (target_n_component - 1)
target_W, target_V_pca = np.linalg.eig(target_cov)

print(target_W.shape)
print(target_V_pca.shape)

# データを主成分の空間に変換する
target_X_pca = target_X_std @ target_V_pca
print(target_X_pca.shape)

eigenvectors = source_X_pca.reshape((Mixtured, D, S)), target_X_pca.reshape((Mixtured, D, S))
source_bias = np.mean(sv[:, :, :D], axis=0)
target_bias = np.mean(sv[:, :, D:], axis=0)
biasvectors = source_bias.reshape((Mixtured, D)), target_bias.reshape((Mixtured, D))

print("Do PCA time = ", time.time() - timer_start , "[sec]")  

(960,)
(960, 960)
(1012, 960)
(960,)
(960, 960)
(1012, 960)
Do PCA time =  3.2743709087371826 [sec]


In [10]:
"""
声質変換に用いる変換元音声と目標音声を読み込む.
"""

timer_start = time.time()
source_mfcc_for_convert = []
source_sp_for_convert = []
source_f0_for_convert = []
source_ap_for_convert = []
fs_source = None
for name in sorted(glob.iglob(for_convert_source, recursive=True)):
    print("source = ", name)
    x_source, fs_source = sf.read(name)
    f0_source, sp_source, ap_source = analyse_by_world_with_harverst(x_source, fs_source)
    mfcc_source = MFCC(fs_source)
    #mfcc_s_tmp = mfcc_s.mfcc(sp)
    #source_mfcc_for_convert = np.hstack([mfcc_s_tmp, mfcc_s.delta(mfcc_s_tmp)])
    source_mfcc_for_convert.append(mfcc_source.mfcc(sp_source))
    source_sp_for_convert.append(sp_source)
    source_f0_for_convert.append(f0_source)
    source_ap_for_convert.append(ap_source)

target_mfcc_for_fit = []
target_f0_for_fit = []
target_ap_for_fit = []
for name in sorted(glob.iglob(for_convert_target, recursive=True)):
    print("target = ", name)
    x_target, fs_target = sf.read(name)
    f0_target, sp_target, ap_target = analyse_by_world_with_harverst(x_target, fs_target)
    mfcc_target = MFCC(fs_target)
    #mfcc_target_tmp = mfcc_target.mfcc(sp_target)
    #target_mfcc_for_fit = np.hstack([mfcc_t_tmp, mfcc_t.delta(mfcc_t_tmp)])
    target_mfcc_for_fit.append(mfcc_target.mfcc(sp_target))
    target_f0_for_fit.append(f0_target)
    target_ap_for_fit.append(ap_target)

# 全部numpy.arrrayにしておく
source_data_mfcc = np.array(source_mfcc_for_convert)
source_data_sp = np.array(source_sp_for_convert)
source_data_f0 = np.array(source_f0_for_convert)
source_data_ap = np.array(source_ap_for_convert)

target_mfcc = np.array(target_mfcc_for_fit)
target_f0 = np.array(target_f0_for_fit)
target_ap = np.array(target_ap_for_fit)

print("Load Input and Target Voice time = ", time.time() - timer_start , "[sec]")  

source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A11.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A14.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A17.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A18.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A19.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A20.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A21.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A22.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A23.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A24.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A25.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A26.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A27.wav
source =  ./

In [11]:
"""
適応話者学習を行う.
つまり,事前学習出力話者から目標話者の空間を作りだす.

適応話者文数ごとにfitted_targetを集めるのは未実装.
"""

timer_start = time.time()
epoch=1000

py = GMM(n_components = Mixtured, covariance_type = 'full')
py.weights_ = weights
py.means_ = target_means
py.covars_ = covarYY

fitted_target = None

for i in range(len(target_mfcc)):
    print("adaptation = ", i+1, "/", len(target_mfcc))
    target = target_mfcc[i]

    for x in range(epoch):
        print("epoch = ", x)
        predict = py.predict_proba(np.atleast_2d(target))
        y = np.sum([predict[:, i: i + 1] * (target - biasvectors[1][i])
                    for i in range(Mixtured)], axis = 1)
        gamma = np.sum(predict, axis = 0)

        left = np.sum([gamma[i] * np.dot(eigenvectors[1][i].T,
                                         np.linalg.solve(py.covars_, eigenvectors[1])[i])
                       for i in range(Mixtured)], axis=0)
        right = np.sum([np.dot(eigenvectors[1][i].T, 
                               np.linalg.solve(py.covars_, y)[i]) 
                        for i in range(Mixtured)], axis = 0)
        weight = np.linalg.solve(left, right)

        fitted_target = np.dot(eigenvectors[1], weight) + biasvectors[1]
        py.means_ = fitted_target
        
print("Load Input and Target Voice time = ", time.time() - timer_start , "[sec]")  

adaptation =  1 / 31
epoch =  0
epoch =  1
epoch =  2
epoch =  3
epoch =  4
epoch =  5
epoch =  6
epoch =  7
epoch =  8
epoch =  9
epoch =  10
epoch =  11
epoch =  12
epoch =  13
epoch =  14
epoch =  15
epoch =  16
epoch =  17
epoch =  18
epoch =  19
epoch =  20
epoch =  21
epoch =  22
epoch =  23
epoch =  24
epoch =  25
epoch =  26
epoch =  27
epoch =  28
epoch =  29
epoch =  30
epoch =  31
epoch =  32
epoch =  33
epoch =  34
epoch =  35
epoch =  36
epoch =  37
epoch =  38
epoch =  39
epoch =  40
epoch =  41
epoch =  42
epoch =  43
epoch =  44
epoch =  45
epoch =  46
epoch =  47
epoch =  48
epoch =  49
epoch =  50
epoch =  51
epoch =  52
epoch =  53
epoch =  54
epoch =  55
epoch =  56
epoch =  57
epoch =  58
epoch =  59
epoch =  60
epoch =  61
epoch =  62
epoch =  63
epoch =  64
epoch =  65
epoch =  66
epoch =  67
epoch =  68
epoch =  69
epoch =  70
epoch =  71
epoch =  72
epoch =  73
epoch =  74
epoch =  75
epoch =  76
epoch =  77
epoch =  78
epoch =  79
epoch =  80
epoch =  81
epoch

epoch =  638
epoch =  639
epoch =  640
epoch =  641
epoch =  642
epoch =  643
epoch =  644
epoch =  645
epoch =  646
epoch =  647
epoch =  648
epoch =  649
epoch =  650
epoch =  651
epoch =  652
epoch =  653
epoch =  654
epoch =  655
epoch =  656
epoch =  657
epoch =  658
epoch =  659
epoch =  660
epoch =  661
epoch =  662
epoch =  663
epoch =  664
epoch =  665
epoch =  666
epoch =  667
epoch =  668
epoch =  669
epoch =  670
epoch =  671
epoch =  672
epoch =  673
epoch =  674
epoch =  675
epoch =  676
epoch =  677
epoch =  678
epoch =  679
epoch =  680
epoch =  681
epoch =  682
epoch =  683
epoch =  684
epoch =  685
epoch =  686
epoch =  687
epoch =  688
epoch =  689
epoch =  690
epoch =  691
epoch =  692
epoch =  693
epoch =  694
epoch =  695
epoch =  696
epoch =  697
epoch =  698
epoch =  699
epoch =  700
epoch =  701
epoch =  702
epoch =  703
epoch =  704
epoch =  705
epoch =  706
epoch =  707
epoch =  708
epoch =  709
epoch =  710
epoch =  711
epoch =  712
epoch =  713
epoch =  714

epoch =  276
epoch =  277
epoch =  278
epoch =  279
epoch =  280
epoch =  281
epoch =  282
epoch =  283
epoch =  284
epoch =  285
epoch =  286
epoch =  287
epoch =  288
epoch =  289
epoch =  290
epoch =  291
epoch =  292
epoch =  293
epoch =  294
epoch =  295
epoch =  296
epoch =  297
epoch =  298
epoch =  299
epoch =  300
epoch =  301
epoch =  302
epoch =  303
epoch =  304
epoch =  305
epoch =  306
epoch =  307
epoch =  308
epoch =  309
epoch =  310
epoch =  311
epoch =  312
epoch =  313
epoch =  314
epoch =  315
epoch =  316
epoch =  317
epoch =  318
epoch =  319
epoch =  320
epoch =  321
epoch =  322
epoch =  323
epoch =  324
epoch =  325
epoch =  326
epoch =  327
epoch =  328
epoch =  329
epoch =  330
epoch =  331
epoch =  332
epoch =  333
epoch =  334
epoch =  335
epoch =  336
epoch =  337
epoch =  338
epoch =  339
epoch =  340
epoch =  341
epoch =  342
epoch =  343
epoch =  344
epoch =  345
epoch =  346
epoch =  347
epoch =  348
epoch =  349
epoch =  350
epoch =  351
epoch =  352

epoch =  907
epoch =  908
epoch =  909
epoch =  910
epoch =  911
epoch =  912
epoch =  913
epoch =  914
epoch =  915
epoch =  916
epoch =  917
epoch =  918
epoch =  919
epoch =  920
epoch =  921
epoch =  922
epoch =  923
epoch =  924
epoch =  925
epoch =  926
epoch =  927
epoch =  928
epoch =  929
epoch =  930
epoch =  931
epoch =  932
epoch =  933
epoch =  934
epoch =  935
epoch =  936
epoch =  937
epoch =  938
epoch =  939
epoch =  940
epoch =  941
epoch =  942
epoch =  943
epoch =  944
epoch =  945
epoch =  946
epoch =  947
epoch =  948
epoch =  949
epoch =  950
epoch =  951
epoch =  952
epoch =  953
epoch =  954
epoch =  955
epoch =  956
epoch =  957
epoch =  958
epoch =  959
epoch =  960
epoch =  961
epoch =  962
epoch =  963
epoch =  964
epoch =  965
epoch =  966
epoch =  967
epoch =  968
epoch =  969
epoch =  970
epoch =  971
epoch =  972
epoch =  973
epoch =  974
epoch =  975
epoch =  976
epoch =  977
epoch =  978
epoch =  979
epoch =  980
epoch =  981
epoch =  982
epoch =  983

epoch =  545
epoch =  546
epoch =  547
epoch =  548
epoch =  549
epoch =  550
epoch =  551
epoch =  552
epoch =  553
epoch =  554
epoch =  555
epoch =  556
epoch =  557
epoch =  558
epoch =  559
epoch =  560
epoch =  561
epoch =  562
epoch =  563
epoch =  564
epoch =  565
epoch =  566
epoch =  567
epoch =  568
epoch =  569
epoch =  570
epoch =  571
epoch =  572
epoch =  573
epoch =  574
epoch =  575
epoch =  576
epoch =  577
epoch =  578
epoch =  579
epoch =  580
epoch =  581
epoch =  582
epoch =  583
epoch =  584
epoch =  585
epoch =  586
epoch =  587
epoch =  588
epoch =  589
epoch =  590
epoch =  591
epoch =  592
epoch =  593
epoch =  594
epoch =  595
epoch =  596
epoch =  597
epoch =  598
epoch =  599
epoch =  600
epoch =  601
epoch =  602
epoch =  603
epoch =  604
epoch =  605
epoch =  606
epoch =  607
epoch =  608
epoch =  609
epoch =  610
epoch =  611
epoch =  612
epoch =  613
epoch =  614
epoch =  615
epoch =  616
epoch =  617
epoch =  618
epoch =  619
epoch =  620
epoch =  621

...

epoch =  941
epoch =  942
epoch =  943
epoch =  944
epoch =  945
epoch =  946
epoch =  947
epoch =  948
epoch =  949
epoch =  950
epoch =  951
epoch =  952
epoch =  953
epoch =  954
epoch =  955
epoch =  956
epoch =  957
epoch =  958
epoch =  959
epoch =  960
epoch =  961
epoch =  962
epoch =  963
epoch =  964
epoch =  965
epoch =  966
epoch =  967
epoch =  968
epoch =  969
epoch =  970
epoch =  971
epoch =  972
epoch =  973
epoch =  974
epoch =  975
epoch =  976
epoch =  977
epoch =  978
epoch =  979
epoch =  980
epoch =  981
epoch =  982
epoch =  983
epoch =  984
epoch =  985
epoch =  986
epoch =  987
epoch =  988
epoch =  989
epoch =  990
epoch =  991
epoch =  992
epoch =  993
epoch =  994
epoch =  995
epoch =  996
epoch =  997
epoch =  998
epoch =  999
adaptation =  29 / 31
epoch =  0
epoch =  1
epoch =  2
epoch =  3
epoch =  4
epoch =  5
epoch =  6
epoch =  7
epoch =  8
epoch =  9
epoch =  10
epoch =  11
epoch =  12
epoch =  13
epoch =  14
epoch =  15
epoch =  16
epoch =  17
epoch

epoch =  578
epoch =  579
epoch =  580
epoch =  581
epoch =  582
epoch =  583
epoch =  584
epoch =  585
epoch =  586
epoch =  587
epoch =  588
epoch =  589
epoch =  590
epoch =  591
epoch =  592
epoch =  593
epoch =  594
epoch =  595
epoch =  596
epoch =  597
epoch =  598
epoch =  599
epoch =  600
epoch =  601
epoch =  602
epoch =  603
epoch =  604
epoch =  605
epoch =  606
epoch =  607
epoch =  608
epoch =  609
epoch =  610
epoch =  611
epoch =  612
epoch =  613
epoch =  614
epoch =  615
epoch =  616
epoch =  617
epoch =  618
epoch =  619
epoch =  620
epoch =  621
epoch =  622
epoch =  623
epoch =  624
epoch =  625
epoch =  626
epoch =  627
epoch =  628
epoch =  629
epoch =  630
epoch =  631
epoch =  632
epoch =  633
epoch =  634
epoch =  635
epoch =  636
epoch =  637
epoch =  638
epoch =  639
epoch =  640
epoch =  641
epoch =  642
epoch =  643
epoch =  644
epoch =  645
epoch =  646
epoch =  647
epoch =  648
epoch =  649
epoch =  650
epoch =  651
epoch =  652
epoch =  653
epoch =  654

epoch =  215
epoch =  216
epoch =  217
epoch =  218
epoch =  219
epoch =  220
epoch =  221
epoch =  222
epoch =  223
epoch =  224
epoch =  225
epoch =  226
epoch =  227
epoch =  228
epoch =  229
epoch =  230
epoch =  231
epoch =  232
epoch =  233
epoch =  234
epoch =  235
epoch =  236
epoch =  237
epoch =  238
epoch =  239
epoch =  240
epoch =  241
epoch =  242
epoch =  243
epoch =  244
epoch =  245
epoch =  246
epoch =  247
epoch =  248
epoch =  249
epoch =  250
epoch =  251
epoch =  252
epoch =  253
epoch =  254
epoch =  255
epoch =  256
epoch =  257
epoch =  258
epoch =  259
epoch =  260
epoch =  261
epoch =  262
epoch =  263
epoch =  264
epoch =  265
epoch =  266
epoch =  267
epoch =  268
epoch =  269
epoch =  270
epoch =  271
epoch =  272
epoch =  273
epoch =  274
epoch =  275
epoch =  276
epoch =  277
epoch =  278
epoch =  279
epoch =  280
epoch =  281
epoch =  282
epoch =  283
epoch =  284
epoch =  285
epoch =  286
epoch =  287
epoch =  288
epoch =  289
epoch =  290
epoch =  291

epoch =  846
epoch =  847
epoch =  848
epoch =  849
epoch =  850
epoch =  851
epoch =  852
epoch =  853
epoch =  854
epoch =  855
epoch =  856
epoch =  857
epoch =  858
epoch =  859
epoch =  860
epoch =  861
epoch =  862
epoch =  863
epoch =  864
epoch =  865
epoch =  866
epoch =  867
epoch =  868
epoch =  869
epoch =  870
epoch =  871
epoch =  872
epoch =  873
epoch =  874
epoch =  875
epoch =  876
epoch =  877
epoch =  878
epoch =  879
epoch =  880
epoch =  881
epoch =  882
epoch =  883
epoch =  884
epoch =  885
epoch =  886
epoch =  887
epoch =  888
epoch =  889
epoch =  890
epoch =  891
epoch =  892
epoch =  893
epoch =  894
epoch =  895
epoch =  896
epoch =  897
epoch =  898
epoch =  899
epoch =  900
epoch =  901
epoch =  902
epoch =  903
epoch =  904
epoch =  905
epoch =  906
epoch =  907
epoch =  908
epoch =  909
epoch =  910
epoch =  911
epoch =  912
epoch =  913
epoch =  914
epoch =  915
epoch =  916
epoch =  917
epoch =  918
epoch =  919
epoch =  920
epoch =  921
epoch =  922

epoch =  483
epoch =  484
epoch =  485
epoch =  486
epoch =  487
epoch =  488
epoch =  489
epoch =  490
epoch =  491
epoch =  492
epoch =  493
epoch =  494
epoch =  495
epoch =  496
epoch =  497
epoch =  498
epoch =  499
epoch =  500
epoch =  501
epoch =  502
epoch =  503
epoch =  504
epoch =  505
epoch =  506
epoch =  507
epoch =  508
epoch =  509
epoch =  510
epoch =  511
epoch =  512
epoch =  513
epoch =  514
epoch =  515
epoch =  516
epoch =  517
epoch =  518
epoch =  519
epoch =  520
epoch =  521
epoch =  522
epoch =  523
epoch =  524
epoch =  525
epoch =  526
epoch =  527
epoch =  528
epoch =  529
epoch =  530
epoch =  531
epoch =  532
epoch =  533
epoch =  534
epoch =  535
epoch =  536
epoch =  537
epoch =  538
epoch =  539
epoch =  540
epoch =  541
epoch =  542
epoch =  543
epoch =  544
epoch =  545
epoch =  546
epoch =  547
epoch =  548
epoch =  549
epoch =  550
epoch =  551
epoch =  552
epoch =  553
epoch =  554
epoch =  555
epoch =  556
epoch =  557
epoch =  558
epoch =  559

In [12]:
"""
変換に必要なものを残しておく.
"""
np.save(save_for_evgmm_covarXX, covarXX)
np.save(save_for_evgmm_covarYX, covarYX)
np.save(save_for_evgmm_fitted_source, fitted_source)
np.save(save_for_evgmm_fitted_target, fitted_target)
np.save(save_for_evgmm_weights, weights)
np.save(save_for_evgmm_source_means, source_means)